In [1]:
import pandas as pd
import numpy as np

In [2]:
#Read The DataSet from cuurent Folder
car = pd.read_csv('quikr_car.csv')

In [3]:
#See the Heads of dataset
car.head()

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,"80,000","45,000 kms",Petrol
1,Mahindra Jeep CL550 MDI,Mahindra,2006,"4,25,000",40 kms,Diesel
2,Maruti Suzuki Alto 800 Vxi,Maruti,2018,Ask For Price,"22,000 kms",Petrol
3,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,"3,25,000","28,000 kms",Petrol
4,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,"5,75,000","36,000 kms",Diesel


In [4]:
#Check the shape of data
car.shape

(892, 6)

In [5]:
#Lets See More info about Dataset like types
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        892 non-null    object
 1   company     892 non-null    object
 2   year        892 non-null    object
 3   Price       892 non-null    object
 4   kms_driven  840 non-null    object
 5   fuel_type   837 non-null    object
dtypes: object(6)
memory usage: 41.9+ KB


**Quality:**


*   Year has many non_year values
*   Year object to Int
*   Price has Ask for price
*   Price object to Int
*   Kms driven has Kms with Integers
*   Kms Driven object to Int
*   Kms Driven has Nan Values
*   Fuel Types has nan values
*   Keep only first three words of name





**Cleaning**

In [6]:
#Star by Creating a backup
backup = car.copy()

In [7]:
#Remove Non Numeric Values
car = car[car["year"].str.isnumeric()]

In [8]:
#Convet object into Int
car["year"]= car["year"].astype(int)

In [9]:
#Remove Ask for Price In Price Colum
car = car[car["Price"] != "Ask For Price"]

In [10]:
#Remoce Commas and Convect Object into Int
car["Price"] = car["Price"].str.replace(",","").astype(int)

In [11]:
#Remove Kms and Cooma
car["kms_driven"] = car["kms_driven"].str.split().str.get(0).str.replace(",","")

In [12]:
#Rmove Non Numeric Values
car = car[car["kms_driven"].str.isnumeric()]

In [13]:
#Convert Object into Int
car["kms_driven"]= car["kms_driven"].astype(int)

In [14]:
#Remove the nan value
car = car[~car["fuel_type"].isna()]

In [15]:
#Get First Three  words of Name
car['name'] = car["name"].str.split(" ").str.slice(0,3).str.join(" ")

In [16]:
#Lets Reset The Index
car = car.reset_index(drop=True)

In [17]:
#Now Cleaned Data
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816 entries, 0 to 815
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        816 non-null    object
 1   company     816 non-null    object
 2   year        816 non-null    int64 
 3   Price       816 non-null    int64 
 4   kms_driven  816 non-null    int64 
 5   fuel_type   816 non-null    object
dtypes: int64(3), object(3)
memory usage: 38.4+ KB


In [18]:
car.describe()

,year,Price,kms_driven
count,816.000000,8.160000e+02,816.000000
mean,2012.444853,4.117176e+05,46275.531863
std,4.002992,4.751844e+05,34297.428044
min,1995.000000,3.000000e+04,0.000000
25%,2010.000000,1.750000e+05,27000.000000
50%,2013.000000,2.999990e+05,41000.000000
75%,2015.000000,4.912500e+05,56818.500000
max,2019.000000,8.500003e+06,400000.000000


In [19]:
#Drop the misatken highst value to let our model work efficiently
car=car[car["Price"]< 6e6].reset_index(drop=True)

In [20]:
car.to_csv("CleanedCar.csv")

**Model**

In [21]:
# Make feauture and Output Label
X= car.drop(columns="Price")
y = car["Price"]

In [22]:
# Make Train and Test Split Data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.2)

In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [24]:
#Make One Hot Endoing fro String Values
ohe = OneHotEncoder()
ohe.fit(X[["name","company","fuel_type"]])

OneHotEncoder()

In [25]:
#Tranfrom the column using Column Tranformer
column_trans = make_column_transformer((OneHotEncoder(categories=ohe.categories_),["name","company","fuel_type"]),
                                    remainder="passthrough")

In [26]:
lr = LinearRegression()

In [27]:
#Send Raw data from one end and get prediction from other end
pipe = make_pipeline(column_trans,lr)

In [28]:
#Can use directly this pipeline
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Audi A3 Cabriolet', 'Audi A4 1.8', 'Audi A4 2.0', 'Audi A6 2.0',
       'Audi A8', 'Audi Q3 2.0', 'Audi Q5 2.0', 'Audi Q7', 'BMW 3 Series',
       'BMW 5 Series', 'BMW 7 Series', 'BMW X1', 'BMW X1 sDrive20d',
       'BMW X1 xDrive20d', 'Chevrolet Beat', 'Chevrolet Beat...
                                                                            array(['Audi', 'BMW', 'Chevrolet', 'Datsun', 'Fiat', 'Force', 'Ford',
       'Hindustan', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Land',
       'Mahindra', 'Maruti', 'Mercedes', 'Mini', 'Mitsubishi', 'Nissan',
       'Renault', 'Skoda', 'Tata', 'Toyota', 'Volkswagen', 'Volvo'],
      dtype=object),
                                                                            array(['Diesel', 'LPG', 'Petrol'], dtype=object)]),
                                                  ['name', 'company',
                                                   'fuel_type'])])),
                ('linearregression', LinearRegression())])

In [29]:
#     Lets Get the prediction on Test Set
y_pred = pipe.predict(X_test)

In [30]:
#Check the r2 score
r2_score(y_test,y_pred)

0.5919464887481327

In [34]:
#Lets Get Maximum R2 Score
scores=[]
for i in range(5000):
  X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=i)
  lr = LinearRegression()
  pipe = make_pipeline(column_trans,lr)
  pipe.fit(X_train,y_train)
  y_pred = pipe.predict(X_test)
  scores.append(r2_score(y_test,y_pred))

In [35]:
np.argmax(scores)

2711

In [36]:
scores[np.argmax(scores)]

0.8639294570456644

In [37]:
#Choose the highest R2 Scoree Model
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=np.argmax(scores))
lr = LinearRegression()
pipe = make_pipeline(column_trans,lr)
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
r2_score(y_test,y_pred)

0.8639294570456644

**Testing An Input AND Dump file using pickle**

In [38]:
import pickle

In [39]:
pickle.dump(pipe,open("LinearRegressionModel.pkl","wb"))

In [40]:
pipe.predict(pd.DataFrame([["Maruti Suzuki Swift","Maruti",2019,100,"Petrol"]],columns=["name","company","year","kms_driven","fuel_type"]))

array([465796.63941281])

Download Both pkl and Cleaned Car Csv to use in Flask App